## Select HNE directory for site

In [ ]:
using CSV
using Dates, DataFrames, DSP
using LaTeXStrings
using NativeFileDialog
using Plots, Printf
using Statistics
using Tk

################################################
################################################
################################################
##           START OF MAIN PROGRAM
################################################
################################################
################################################

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")
# Pick directory containing Triaxys HNE .txt files
###triaxys_directory = pick_folder("C:\\QGHL\\Wave_data\\")
triaxys_directory = pick_folder()

# build list of all triaxys files in selected directory
triaxys_files = filter(x->occursin("_HNE_",x), readdir(triaxys_directory));
triaxys_files = triaxys_files[findall(x->endswith(uppercase(x), ".TXT"), triaxys_files)]

# Check whether any triaxys files exist in selected directory. If not, EXIT!
if length(triaxys_files) == 0
    println("No triaxys files found in "*triaxys_directory)
    exit;
else
    println(string(length(triaxys_files)) * " HNE files found")
end

## Process all HNE files for selected month

In [ ]:
# create a df that will hold all the WSE data from the HNE files in selected monthly directory
HNE_df = DataFrame(Date = [], Heave = [], North = [], East = [])

println("BE WARNED - THIS WILL TAKE A COUPLE OF MINUTES!\n")
flush(stdout)

for i in triaxys_files
    
    # Read the header information contained in the first 11 rows   
##    println("Processing " * i)
    hne_file = triaxys_directory * "//" * i
    
    # Check whether file is empty
    if (stat(hne_file).size > 0) 
        
        fp = open(triaxys_directory * "//" * i)
        
        data_report = rsplit(readline(fp), "- ")[2]
        version = rsplit(readline(fp), "= ")[2]
        block = rsplit(readline(fp), "= ")[2]
        date = Dates.DateTime(rsplit(rsplit(readline(fp),"= ")[2],"(")[1],"yyyy-mm-dd HH:MM")
        num_points = parse(Int,rsplit(readline(fp), "= ")[2])
        first_point = parse(Float64,rsplit(readline(fp), "= ")[2])
        interval = parse(Float64,rsplit(readline(fp), "= ")[2])

        close(fp)
##        println(date,' ',data_report,' ',version,' ',block,' ',num_points,' ',first_point,' ',interval)
        
        if num_points > 0    # Ignore those HNE files without WSE data
            # get the WSEs from each HNE file and append to df
            df = DataFrame(CSV.File(triaxys_directory * "//" * i; header=false, skipto=12, delim=" ",ignorerepeated=true));

            # convert time in seconds to date
            df[!,:Column1] = date + Microsecond.(trunc.(Int64,df[!,:Column1] .* 1000000));
            colnames = ["Date","Heave","North","East"]
            rename!(df, Symbol.(colnames))

            # add HNE files WSE data to dataframe
            HNE_df = vcat(HNE_df, df)
        end
            
    else

        println("Alert - " * i * " is an empty file!")

    end
        
end

## Select and plot a 30-minute record

In [ ]:
function plot_hne(record_df, first_date, last_date)
######################################## 

    function spike_value(wse)
    #####################################    
        median_value = median(wse)
        std_value = std(wse)
        
        return(median_value + 3*std_value)
        
        end    # spike_value()


    # get WSEs for desired 30-minute record
    heave = record_df.Heave
    north = record_df.North
    east = -record_df.East    # <<<==================== This function is adapted from my HXV version (which assumes positive-West), hence reverse of sign for East

    spike = spike_value(heave)
    heave_spikes = findall(i->(i>=spike), abs.(heave));

    spike = spike_value(north)
    north_spikes = findall(i->(i>=spike), abs.(north));

    spike = spike_value(east)
    east_spikes = findall(i->(i>=spike), abs.(east));

    times = record_df.Date

    # create plots of heave, north, and east
    title_string = Dates.format(first_date, "dd/mm/yyyy HH:MM")
    p1_hne = scatter(times[heave_spikes], heave[heave_spikes], label="", markershape=:circle, ms=4, mc=:white, ma=1, msc=:red, msa=0.25, msw=0.5)
    p1_hne = plot!(times,heave, label="", c="#4a536b", lw=0.5, title=title_string, titlefontsize=12) ##last(split(infil,"\\")))

    # get plotting limits
    x_lim1 = xlims(p1_hne)[1]; y_lim1 = ylims(p1_hne)[1]
    x_lim2 = xlims(p1_hne)[2]; y_lim2 = ylims(p1_hne)[2]

    p2_hne = scatter(times[north_spikes], north[north_spikes], label="", markershape=:circle, ms=4, mc=:white, ma=1, msc=:red, msa=0.25, msw=0.5)
    p2_hne = plot!(times,north, label="", c="#aed6dc", lw=0.5)
    
    p3_hne = scatter(times[east_spikes], east[east_spikes], label="", markershape=:circle, ms=4, mc=:white, ma=1, msc=:red, msa=0.25, msw=0.5)
    p3_hne = plot!(times,east, label="", c="#ff9a8d", lw=0.5)

    hline!(p1_hne, [0], lw=1, label="")
    hline!(p2_hne, [0], lw=1, label="")
    hline!(p3_hne, [0], lw=1, label="")

    # get plotting limits
    x_lim1 = xlims(p1_hne)[1]; y_lim1 = ylims(p1_hne)[1]
    x_lim2 = xlims(p1_hne)[2]; y_lim2 = ylims(p1_hne)[2]

    # display plots to screen
    # display plots to screen
    tm_tick = range(first_date,last_date,step=Minute(2))
    ticks = Dates.format.(tm_tick,"MM:SS")

    plot_wse = Plots.plot(p1_hne, p2_hne, p3_hne, xticks=(tm_tick,ticks), layout = (3, 1), size = (1400, 600),
        xlims=(first_date,last_date),  xtickfontsize=7, ytickfontsize=8,
        framestyle = :box, legend=:bottomleft,
        margin = 1Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)            

    display(plot_wse)

    # create a plot file to be saved as a .PNG
##    plt_file = first(infil, length(infil)-4)*"_plot_hne_"*Dates.format(start_date, "yyyy_mm_dd_HHMM")*".png"

    # Save plot to file
##    savefig(plt_file)
##    println("Plot file saved as ",plt_file)
       
    end    # plot_hne()


function plot_spectra(record_df)
################################################    
    Sample_frequency = 1.28

    function calc_freq_pden(wse)
        ps_w = DSP.Periodograms.welch_pgram(float(wse), 512, 256; onesided=true, nfft=512, fs=Sample_frequency, window=hanning);
        freqs = freq(ps_w);
        pden = power(ps_w);

        return(freqs, pden)

        end

    heave = float.(record_df.Heave)
    north = float.(record_df.North)
    east = -float.(record_df.East)    # <<<==================== This function is adapted from my HXV version (which assumes positive-West), hence reverse of sign for East

    heave_freq, heave_pden = calc_freq_pden(heave);
    north_freq, north_pden = calc_freq_pden(north);
    east_freq, east_pden = calc_freq_pden(east);

    title_string = Dates.format(first(record_df.Date), "dd/mm/yyyy HH:MM")

    max_y = max(heave_pden[argmax(heave_pden)], north_pden[argmax(north_pden)], east_pden[argmax(east_pden)])

    # Plot the spectra for heave, North, and East
    spectra = plot(heave_freq,heave_pden, c=:red, lw=1, alpha=.5, fillrange = 0, fillalpha = 0.015, fillcolor = :red, label="Heave", xlim=(0,1.0), ylim=(0,max_y*1.05),
        xticks = (0.0:0.1:1.0, string.(0.0:0.1:1.0)), xtickfontsize=7, xlabel="Frequency (Hertz)",
        ytickfontsize=8, ylabel="Spectral Density (m"*L"^2"*"/Hz.)")
    spectra = plot!(north_freq,north_pden, c=:green, lw=1, alpha=.5, fillrange = 0, fillalpha = 0.015, fillcolor = :green, label="North")
    spectra = plot!(east_freq,east_pden, c=:blue, lw=1, alpha=.5, fillrange = 0, fillalpha = 0.015, fillcolor = :red,  label="East")
    spectra = vline!([0.05; 0.05], lw=2, ls =:dash, c=:red, label="")

    # Plot scatter of East-East and North-South buoy movement
    p1 = plot(east,north, zcolor=heave, m=(1, 3, :RdYlGn_11, Plots.stroke(0)), leg=false, cbar=false, c="lightgrey", label="",
        aspect_ratio=:equal,
            w=0.5, title="East-North", titlefontsize=10, xlabel="East-East displacement (m)", ylabel="North-South displacement (m)")
    
    spectral_plot = plot(spectra, p1, layout = Plots.grid(1, 2, widths=(5/8,3/8)), size = (1400, 600), framestyle = :box,
        fg_legend=:transparent, bg_legend=:transparent, legend=:topright, foreground_color_grid="lightgrey", 
        xtickfontsize=8, ytickfontsize=8, xguidefontsize=10, yguidefontsize=10,
        grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, show=true, margin = 10Plots.mm) 

    display(spectral_plot)

    return()

    end    # plot_spectra()


# Determine first and last record times
start_date = floor(first(HNE_df.Date), Dates.Minute(30));
finish_date = ceil(last(HNE_df.Date), Dates.Minute(30)) - Minute(30);

##available_dates = collect(start_date:Minute(30):finish_date) # Possible now superceded by lines below

# Select only 30-minute records that have some data
available_records = DateTime.(unique!(hcat(split.(readdir(triaxys_directory),"_")...)[1, :]), "yyyymmddHHMMSS")
available_dates = unique!(floor.(available_records, Dates.Minute(30)))
dates = Dates.format.(available_dates, "yyyy-mm-dd HH:MM")

w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, dates)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    global date_choice = get_value(lb);
    
    global first_date = DateTime(date_choice[1], dateformat"y-m-d H:M")
    global last_date = first_date + Minute.(30)

##    println(first_date,' ',last_date)
##    flush(stdout)
    global record_df = HNE_df[first_date .<= HNE_df.Date .< last_date, :]
    
    # Check whether data is available for selected 30-minute record
    
##    println(isempty(record_df),' ',nrow(record_df))
##    if !isempty(record_df)
    if nrow(record_df) > 0
    
        plot_hne(record_df, first_date, last_date)
##        plot_spectra(record_df)
    
    else
        
        println("Alert: No data available between " * Dates.format(first_date, "dd/mm/yyyy HH:MM") * " and " * Dates.format(last_date, "dd/mm/yyyy HH:MM"))
        flush(stdout)
        
    end

end

## Investigate calculating cross-spectra (in order to calculate Fourier coefficients)

In [ ]:
using FFTW
using DSP


function calc_csd_welch(X,Y,sample_frequency,len,olap)
#####################################
# calculate the cross power spectra between X and Y
    
    segmentsX = arraysplit(X,len,olap)
    segmentsY = arraysplit(Y,len,olap)

#    N = length(X)
    dt = 1/sample_frequency
    
    csd = []

    for i in eachindex(segmentsX)
        
        x = fftshift(fft(segmentsX[i] )) 
        y = fftshift(fft(segmentsY[i] ))

        push!(csd, (2 * dt^2 / len) .* (x .* conj(y)) .* sample_frequency .* DSP.Windows.tukey(256,66/256))

    end
        
    return(mean(csd,dims=1)[1])
    
end    # calc_csd_welch()


N = length(record_df.Heave)
sample_frequency = 1.28
dt = 1/sample_frequency
##tukey = DSP.Windows.tukey(256,66/256)

record_df.Heave = float.(record_df.Heave)
record_df.North = float.(record_df.North)
record_df.East = -float.(record_df.East)        # <<<==================== This function is adapted from my HXV version (which assumes positive-West), hence reverse of sign for East

csd_vv = calc_csd_welch(record_df.Heave,record_df.Heave,sample_frequency,256,0)
csd_nn = calc_csd_welch(record_df.North,record_df.North,sample_frequency,256,0)
csd_ww = calc_csd_welch(record_df.East,record_df.East,sample_frequency,256,0)
csd_nw = calc_csd_welch(record_df.North,record_df.East,sample_frequency,256,0)
csd_vn = calc_csd_welch(record_df.Heave,record_df.North,sample_frequency,256,0)
csd_vw = calc_csd_welch(record_df.Heave,record_df.East,sample_frequency,256,0)

f2 = fftshift(fftfreq(length(csd_vv), 1/dt))
f2_pos_vals = findall( x -> x >= 0, f2 )

# Calculate the co- and quad- spectra
cvv = real(csd_vv[f2_pos_vals])
cnn = real(csd_nn[f2_pos_vals])
cww = real(csd_ww[f2_pos_vals])
cnw = real(csd_nw[f2_pos_vals])
qvn = imag(csd_vn[f2_pos_vals])
qvw = imag(csd_vw[f2_pos_vals])

# Calculate the Fourier Coefficients
a1 = qvn ./ (cvv .* (cnn .+ cww)).^0.5
b1 = qvw ./ (cvv .* (cnn .+ cww)).^0.5

a2 = (cnn .- cww) ./ (cnn .+ cww)
b2 = (2 .* cnw) ./ (cnn .+ cww);

# check for any Nans in the data
replace!(a1, NaN=>0)
replace!(b1, NaN=>0)
replace!(a2, NaN=>0)
replace!(b2, NaN=>0);

# Calculate the Centred Fourier Coefficients
θ₀ = atan.(b1,a1)
m1 = (a1.^2 .+ b1.^2).^0.5
m2 = a2.*cos.(2*θ₀) .+ b2.*sin.(2*θ₀)
n2 = -a2.*sin.(2*θ₀) .+ b2.*cos.(2*θ₀)

# Calculate the spread
σc = (2 .* (1 .- m1)).^0.5;

N = length(record_df.Heave)
f1 = 0.005:0.005:0.64

println(first_date)

p1 = plot(f1,a1, lw=2, c=:blue, label="a₁", fg_legend = :false, title=("QGHL a₁ and b₁"))
p1 = plot!(f1,b1, lw=2, c=:red, label="b₁")

p2 = plot(f1,a2, lw=2, c=:blue, label="a₂", fg_legend = :false, title=("QGHL a₂ and b₂"))
p2 = plot!(f1,b2, lw=2, c=:red, label="b₂")

p3 = plot(f1,m1, lw=2, c=:blue, label="m₁", fg_legend = :false, title=("QGHL m₁ and m₂"))
p3 = plot!(f1,m2, lw=2, c=:red, label="m₂")

p4 = plot(f1,n2, lw=2, c=:blue, label="n₂", fg_legend = :false, title=("QGHL n₂"))

coefficients_plot = plot(p1, p2, p3, p4, layout = (2,2), framestyle = :box, titlefontsize=10, xlabel="Frequency (Hz)", xlabelfontsize=9,
        leftmargin = 12Plots.mm, bottommargin = 12Plots.mm, grid=true, size=(1400, 800), colorbar=false, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

display(coefficients_plot)

## Calculate spectra, direction, and spread

In [ ]:
using DSP

hanning = DSP.Windows.hanning(256; padding=0, zerophase=false)

##tukey = DSP.Windows.tukey(256,66/256)

direction = (rad2deg.(π/2 .- atan.(b1,a1)) .+ 450).%360
dir1 = direction

dir_type = " Magnetic"

p1 = plot(f2[f2_pos_vals], direction, lw=3, c=:blue, yflip=true, ylim=[0,360], yticks = 0:30:360, grid=true, label="Direction", xlabel="Frequency (Hz)", ylabel="Direction ("*L"^o"*dir_type*")", legend=:topright)
p1 = plot!(f2[f2_pos_vals], direction .+ rad2deg.(σc), fillrange = direction .- rad2deg.(σc), fillalpha = 0.25, lw=0, c = 1, label = "Spread")

max_spec = max(maximum(power(welch_pgram(record_df.Heave, 256, 0; fs=sample_frequency, window=hanning))),maximum(real(csd_vv[f2_pos_vals]))) * 1.05

p1 = plot!(twinx(),freq(welch_pgram(record_df.Heave, 256, 0; fs=sample_frequency, window=hanning)),ylabel="Spectral Density (m"*L"^2"*"/Hz.)",
    power(welch_pgram(record_df.Heave, 256, 0; fs=sample_frequency, window=hanning)),c=:red,lw=3, label="DSP Welch\n", ylim=[0,max_spec], legend=:bottomright)
p1 = plot!(twinx(),f2[f2_pos_vals], real(csd_vv[f2_pos_vals]), lw=2, c=:yellow, label="Welch calc. JW", ylim=[0,max_spec], legend=:bottomright)

p_all = plot(p1, framestyle = :box, xlim=[0,0.64], title="Spectra and Direction",
        leftmargin = 15Plots.mm, rightmargin = 25Plots.mm, bottommargin = 15Plots.mm, size=(1400, 600), colorbar=false, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, fg_legend = :false)

display(p_all)

## Triaxys Fourier coefficients

In [ ]:
spectral_large_file = strip(triaxys_directory, ['H','N','E']) * "SPECTRAL\\" *  Dates.format(first_date, "yyyymmddHHMMSS") * "_SPECTRAL_LARGE.txt"
println("Reading ",spectral_large_file)
fp = open(spectral_large_file)

data_report = rsplit(readline(fp), "- ")[2]
version = rsplit(readline(fp), "= ")[2]

file_type = rsplit(readline(fp), "= ")[2]
date_string = rsplit(readline(fp),"= ")[2]
date = Dates.DateTime(rsplit(date_string,"(")[1],"yyyy-mm-dd HH:MM")

println(date_string * " TRIAXYS BUOY DATA REPORT - " * data_report)
    
duration = parse(Int,rsplit(readline(fp), "= ")[2])
frequency_number = parse(Int,rsplit(readline(fp), "= ")[2])
frequency_range = rsplit(readline(fp), "= ")[2]
frequency_spacing = parse(Float64,rsplit(readline(fp), "= ")[2])

resolvable_frequency_range = rsplit(readline(fp), "= ")[2]

# get limits of frequencies
first_frequency = parse(Float64,rsplit(frequency_range, " TO ")[1])
last_frequency = parse(Float64,rsplit(frequency_range, " TO ")[2])

# get limits of resolvable frequencies
start_frequency = parse(Float64,rsplit(resolvable_frequency_range, " TO ")[1])
end_frequency = parse(Float64,rsplit(resolvable_frequency_range, " TO ")[2])

for i in 1:6
    dummy = readline(fp);
#    println(i,dummy)
end

first_frequency = parse(Float64,rsplit(frequency_range, " TO ")[1])
last_frequency = parse(Float64,rsplit(frequency_range, " TO ")[2])

# get limits of resolvable frequencies
start_frequency = parse(Float64,rsplit(resolvable_frequency_range, " TO ")[1])
end_frequency = parse(Float64,rsplit(resolvable_frequency_range, " TO ")[2])

for j in first_frequency:frequency_spacing:start_frequency-frequency_spacing/2
    dummy = readline(fp);
#    println(j,dummy)
end

# create a df that will hold all the WSE data from the HNE files in selected monthly directory
Spectral_df = DataFrame(Frequency = [], Energy = [], a1 = [], b1 = [], a2 = [], b2 = [])


for k in start_frequency:frequency_spacing:end_frequency
    
    Fourier_coefficients = readline(fp)
    frequency = parse(Float64,rsplit(Fourier_coefficients, "  ")[1])
    energy = parse(Float64,rsplit(Fourier_coefficients, "  ")[2])
    
    fail = 0
    try
        a1 = parse(Float64,rsplit(Fourier_coefficients, "  ")[3])
    catch
        fail = 99
    end
    
    try
        b1 = parse(Float64,rsplit(Fourier_coefficients, "  ")[4])
    catch
        fail = 99
    end
    
    try
        a2 = parse(Float64,rsplit(Fourier_coefficients, "  ")[5])
    catch
        fail = 99
    end
        
    try
        b2 = parse(Float64,rsplit(Fourier_coefficients, "  ")[6])
    catch
        fail = 99
    end
    
#    println(frequency,' ',energy,' ',a1,' ',b1,' ',a2,' ',b2)
    
    if (fail != 99)
        push!(Spectral_df,[frequency,energy,a1,b1,a2,b2])
    end
end

close(fp)

# Calculate the Centred Fourier Coefficients
θ₀ = atan.(Spectral_df.b1,Spectral_df.a1)
m1 = (Spectral_df.a1.^2 .+ Spectral_df.b1.^2).^0.5
m2 = Spectral_df.a2.*cos.(2*θ₀) .+ Spectral_df.b2.*sin.(2*θ₀)
n2 = -Spectral_df.a2.*sin.(2*θ₀) .+ Spectral_df.b2.*cos.(2*θ₀)

# Calculate the spread
σc = (2 .* (1 .- m1)).^0.5;

p1 = plot(Spectral_df.Frequency,Spectral_df.a1, lw=2, c=:blue, label="a₁", fg_legend = :false, title=("Triaxys a₁ and b₁"))
p1 = plot!(Spectral_df.Frequency,Spectral_df.b1, lw=2, c=:red, label="b₁")

p2 = plot(Spectral_df.Frequency,Spectral_df.a2, lw=2, c=:blue, label="a₂", fg_legend = :false, title=("Triaxys a₂ and b₂"))
p2 = plot!(Spectral_df.Frequency,Spectral_df.b2, lw=2, c=:red, label="b₂")

p3 = plot(Spectral_df.Frequency,m1, lw=2, c=:blue, label="m₁", fg_legend = :false, title=("Triaxys m₁ and m₂"))
p3 = plot!(Spectral_df.Frequency,m2, lw=2, c=:red, label="m₂")

p4 = plot(Spectral_df.Frequency,n2, lw=2, c=:blue, label="n₂", fg_legend = :false, title=("Triaxys n₂"))

coefficients_plot = plot(p1, p2, p3, p4, layout = (2,2), framestyle = :box, titlefontsize=10, xlabel="Frequency (Hz)", xlabelfontsize=9,
        leftmargin = 12Plots.mm, bottommargin = 12Plots.mm, grid=true, size=(1400, 800), colorbar=false, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

display(coefficients_plot)
    

## Triaxys spectra, direction, and spread

In [ ]:
KVH_file = rsplit(spectral_large_file, "SPECTRAL")[1] * "MEANDIR_KVH\\" * Dates.format(first_date, "yyyymmddHHMMSS") * ".MEANDIR_KVH"
println("Reading ",KVH_file)

fp = open(KVH_file)

data_report = rsplit(readline(fp), "- ")[2]
version = rsplit(readline(fp), "= ")[2]
file_type = rsplit(readline(fp), "= ")[2]
date_string = rsplit(readline(fp),"= ")[2]
date = Dates.DateTime(rsplit(date_string,"(")[1],"yyyy-mm-dd HH:MM")

println(date_string * " TRIAXYS BUOY DATA REPORT - " * data_report)
    
resolvable_points_number = parse(Int,rsplit(readline(fp), "= ")[2])
initial_resolvable_frequency = parse(Float64,rsplit(readline(fp), "= ")[2])
frequency_spacing = parse(Float64,rsplit(readline(fp), "= ")[2])
resolvable_frequency_range = rsplit(readline(fp), "= ")[2]
# get limits of resolvable frequencies
start_frequency = parse(Float64,rsplit(resolvable_frequency_range, " TO ")[1])
end_frequency = parse(Float64,rsplit(resolvable_frequency_range, " TO ")[2])
weighted_mean_direction = parse(Float64,rsplit(readline(fp), "= ")[2])
weighted_mean_spread = parse(Float64,rsplit(readline(fp), "= ")[2])

for i in 1:4
    dummy = readline(fp);
#    println(i,dummy)
end

# create a df that will hold all the WSE data from the HNE files in selected monthly directory
global kvh_df = DataFrame(Frequency = [], Sf = [], Mean_dir = [], Spread = [])


for k in start_frequency:frequency_spacing:end_frequency
    
    kvh_values = readline(fp)
    frequency = parse(Float64,kvh_values[1:5])
    Sf = parse(Float64,kvh_values[6:21])
    Mean_dir = parse(Float64,kvh_values[22:30])
    Spread = parse(Float64,kvh_values[31:37])
    
#    println(frequency,' ',Sf,' ',Mean_dir,' ',Spread)
    push!(kvh_df,[frequency,Sf,Mean_dir,Spread])
end

close(fp)

direction = (rad2deg.(π/2 .- atan.(b1,a1)) .+ 450).%360
dir1 = direction

dir_type = " Magnetic"

p1 = plot(kvh_df.Frequency, kvh_df.Mean_dir, lw=3, c=:blue, yflip=true, ylim=[0,360], yticks = 0:30:360, grid=true, label="Direction", xlabel="Frequency (Hz)", ylabel="Direction ("*L"^o"*dir_type*")", legend=:topright)
p1 = plot!(kvh_df.Frequency, kvh_df.Mean_dir .+ kvh_df.Spread, fillrange = kvh_df.Mean_dir .- kvh_df.Spread, fillalpha = 0.25, lw=0, c = 1, label = "Spread")

max_spec = max(maximum(power(welch_pgram(record_df.Heave, 256, 0; fs=sample_frequency, window=hanning))),maximum(real(csd_vv[f2_pos_vals]))) * 1.05

p1 = plot!(twinx(),Spectral_df.Frequency,Spectral_df.Energy,ylabel="Spectral Density (m"*L"^2"*"/Hz.)",
     c=:red,lw=3, label="Triaxys spectra", ylim=[0,max_spec], legend=:bottomright)

p_all = plot(p1, framestyle = :box, xlim=[0,0.64], title="Spectra and Direction",
        leftmargin = 15Plots.mm, rightmargin = 25Plots.mm, bottommargin = 15Plots.mm, size=(1400, 600), colorbar=false, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, fg_legend = :false)

display(p_all)

## Heatmap of selected Spectral data

In [ ]:
Spectral_df

## Show individual wave heights

In [ ]:
# get date of .csv file
first_date = first(record_df.Date)
last_date = last(record_df.Date)

println("Preparing plots")
flush(stdout)
title_string = Dates.format.(first_date,"dd/mm/yyyy") * " - " * Dates.format.(last_date,"dd/mm/yyyy")
##title_string = Dates.format(first(df.Date), "dd/mm/yyyy HH:MM") * " to " * Dates.format(last(df.Date), "dd/mm/yyyy HH:MM")
p1_hnw = plot(record_df.Date, record_df.Heave, label="Heave", c="#4a536b", lw=0.5, alpha=0.5, title=title_string, titlefontsize=12) ##last(split(infil,"\\")))
p2_hnw = plot(record_df.Date, record_df.North, label="North", c="#aed6dc", lw=0.5)
p3_hnw = plot(record_df.Date, record_df.East, label="East", c="#ff9a8d", lw=0.5)

# display plots to screen
tm_tick = range(first_date,last_date,step=Hour(1))
ticks = Dates.format.(tm_tick,"HH:MM")
plot_wse = plot(p1_hnw, p2_hnw, p3_hnw, xtick=(tm_tick,ticks), layout = (3, 1), size = (1400, 500),
    xlim=(first_date,last_date), ylim=(0,Inf), xtickfontsize=7,ytickfontsize=8,
    framestyle = :box,fg_legend=:transparent, legend=:topleft,
    margin = 1Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

display(plot_wse)

In [ ]:
function show_waves(heave)
################################################
    zero_up = []
    for i in 2:length(heave)-1
        if (heave[i]*heave[i+1] < 0 && heave[i+1] > 0) || (heave[i] == 0 && heave[i-1] < 0 && heave[i+1] > 0)
            push!(zero_up,i)
        end
    end
    
    ##med = median(heave)
    ##stdev = std(heave)
    ##stdev_3 = med + stdev*3

    global wse_point = 1:1:length(heave)
    wse_1 = plot(wse_point, heave[wse_point], c=:blue, alpha=.5, label = "WSE's", fillrange = 0, fillalpha = 0.0075, fillcolor = :blue)
    wse_1 = scatter!(wse_point, heave[wse_point], c=:white, ms=3, 
        markerstrokecolor=:blue, alpha=0.5, markerstrokewidth=0.5, label="WSE points")
    wse_1 = scatter!(zero_up, heave[zero_up], ms=4, c=:lightgreen, 
        markerstrokecolor=:green, alpha=0.5, series_annotations = text.(zero_up, :bottom, :red, :size, 10), 
        annotationhalign = :hcenter, label="Zero up-cross points")

    # heave Threshold set at 10mm. Refer to Section 9 Wave statistics pp. 9-10 in Datawell Library Manual
    threshold = 0.05

    wse_1 = hline!([threshold; threshold], lw=0.2, ls =:dot, c=:red, label="Threshold ("*L"\pm"*string(threshold)*")\n")
    wse_1 = hline!([-threshold; -threshold], lw=0.2, ls =:dot, c=:red, label="")

    ##wse_1 = hline!([stdev_3; stdev_3], lw=0.2, ls =:dot, c=:green, label="3 sigma")
    ##wse_1 = hline!([-stdev_3; -stdev_3], lw=0.2, ls =:dot,  c=:green, label="")

    wse_plot = plot(wse_1, size = (1500, 500),xlim=(1,500), ylim=(-1.5,1.5), framestyle = :box, 
        fg_legend=:transparent, bg_legend=:transparent, legend=:topright,
        margin = 1Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, show=true)

    display(wse_plot)
    
    return()
    
    end    # show_waves()


function show_zero_x(heave, start_date)
################################################
    wse_point = 1:1:length(heave)
    zero_up = []; valid_zero_up = []

    for i in 2:length(heave)-1
        if (heave[i]*heave[i+1] < 0 && heave[i+1] > 0) || (heave[i] == 0 && heave[i-1] < 0 && heave[i+1] > 0)
            push!(zero_up,i)
        end
    end
    
    # heave Threshold set at 10mm. Refer to Section 9 Wave statistics pp. 9-10 in Datawell Library Manual
    threshold = 0.05

    valid_zero_up = []; crest_points = []; trough_points = []
    i = 1; j = 2

    while j < length(zero_up)-1

        crest = maximum(heave[zero_up[i]:zero_up[j]])
        crest_point = zero_up[i] + argmax(heave[zero_up[i]:zero_up[j]]) - 1
        trough = minimum(heave[crest_point:zero_up[j]])

        # Check that crest higher than threshold AND trough less than threshold - Possible Valid Wave!!
        if (crest > threshold) & (trough < -threshold)
            crest_point = zero_up[i] + argmax(heave[zero_up[i]:zero_up[j]]) - 1
            trough_point = crest_point + argmin(heave[crest_point:zero_up[j]]) - 1

            push!(crest_points,crest_point)
            push!(trough_points,trough_point)

            next_crest = maximum(heave[zero_up[j]:zero_up[j+1]])

            # Check that NEXT crest also exceeds threshold (if so then Valid Wave)
            if (next_crest > threshold)
    ##            println("Crest found at ",crest_point," Trough at ",trough_point)
                push!(valid_zero_up,(zero_up[i],zero_up[j]));
                i = j
            end

        end

        j = j+1

    end

    # Process last recorded wave
    #i = j
    #j = j+1

    crest = maximum(heave[zero_up[i]:zero_up[j]])
    trough = minimum(heave[zero_up[i]:zero_up[j]])

    if (crest > threshold) & (trough < -threshold)

        crest_point = zero_up[i] + argmax(heave[zero_up[i]:zero_up[j]]) - 1
        trough_point = crest_point + argmin(heave[crest_point:zero_up[j]]) - 1
        push!(valid_zero_up,(zero_up[i],zero_up[j]));

    end

    heights = []

    for i in 1:length(valid_zero_up)

        crest = maximum(heave[valid_zero_up[i][1]:valid_zero_up[i][2]]);
        trough = minimum(heave[valid_zero_up[i][1]:valid_zero_up[i][2]]);
        push!(heights,crest - trough)
    ##    @printf("Wave %d = %2.3f\n",i,crest - trough)

    end 

    # Get time-domain height parameters
    sorted_heights = sort(heights, rev=true) # sort heights in reverse order heighestwave to loheave wave
    hmax = maximum(sorted_heights)
    hs = mean(sorted_heights[1:Int(ceil(length(sorted_heights)/3))])
    h10 = mean(sorted_heights[1:Int(ceil(length(sorted_heights) / 10))])
    hmean = mean(sorted_heights)

    @printf("%s; Waves = %3d; Hmean = %4.2fm; Hs = %4.2fm; H10 = %4.2fm; Hmax = %4.2fm\n",Dates.format(start_date, "yyyy-mm-dd HH:MM"),length(heights), hmean, hs, h10, hmax)

    ## Locate the zero-crossing points

    x_point = []
    for i in 1:length(valid_zero_up)
        push!(x_point,valid_zero_up[i][1] + abs(heave[valid_zero_up[i][1]]) / (heave[valid_zero_up[i][1]+1] - heave[valid_zero_up[i][1]]))
    end

    # Process final zero-crossing point
    i = length(valid_zero_up)
    push!(x_point,valid_zero_up[i][2] + abs(heave[valid_zero_up[i][2]]) / (heave[valid_zero_up[i][2]+1] - heave[valid_zero_up[i][2]]))

    # Do plots
    wse_1 = plot(wse_point, heave[wse_point], c=:blue, alpha=0.5, label = "WSE's", fillrange = 0, fillalpha = 0.0075, fillcolor = :blue)
    wse_1 = scatter!(wse_point, heave[wse_point], c=:white, ms=3, 
        markerstrokecolor=:blue, alpha=0.5, markerstrokewidth=0.5,label="WSE points")
    wse_1 = scatter!(zero_up, heave[zero_up], ms=3, c=:lightgreen, 
        markerstrokecolor=:lightgreen, series_annotations = text.(zero_up, :bottom, :red, :size, 10), 
        annotationhalign = :hcenter, label="Zero up-cross points\n")
    wse_1 = scatter!(x_point, zeros(length(x_point)), c=:yellow, ms=5, 
        markerstrokecolor=:yellow, markershape=:diamond, label="Zero-crossing points")

    # heave Threshold set at 10mm. Refer to Section 9 Wave statistics pp. 9-10 in Datawell Library Manual
    threshold = 0.05 

    wse_1 = hline!([threshold; threshold], lw=0.2, ls =:dot, c=:red, label="Threshold\n")
    wse_1 = hline!([-threshold; -threshold], lw=0.2, ls =:dot, c=:red, label="")

    wse_plot = plot(wse_1, size = (1500, 500),xlim=(1,500), ylim=(-1.5,1.5), framestyle = :box, 
        fg_legend=:transparent, bg_legend=:transparent, legend=:topright,
        margin = 1Plots.mm, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, show=true)

    display(wse_plot)
    
    return()

    end    # show zero_x()


w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, dates)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(f, "Ok")
pack(b)

bind(b, "command") do path
    
    date_choice = get_value(lb);
    
    first_date = DateTime(date_choice[1], dateformat"y-m-d H:M:S")
    last_date = first_date + Minute.(30)

##    println(date_choice,' ',first_date,' ',last_date)
##    println(first_date,' ',last_date)
    flush(stdout)
    record_df = HNE_df[first_date .<= HNE_df.Date .< last_date, :]
    
    zero_up = []; valid_zero_up = []
    global heave = record_df.Heave
    
##    show_waves(heave)
    show_zero_x(heave, record_df.Date[1])
    
end